Design of a new game, Spuds

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
end

const MXS = 100
const MXV = 20
const MNV = 1

function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
    #return h*(f+l-2*MNV) + f*(p+r+s-3*MNV) + h+f+l+p+r+s-6*MNV
    return h*(f+l) + f*(p+r+s)
end

cost (generic function with 1 method)

In [49]:
function eval_battle(a::Spud, b::Spud)::Int
    a_finds = a.f >= b.h
    b_finds = b.f >= a.h
    #score_a = 4 * (1000*a.p + a.l > 1000*b.p+b.l) + 3 * (1000*a.r+a.l > 1000*b.r+b.l) + 2 * (1000*a.s+a.l > 1000*b.s+b.l)
    #score_b = 4 * (1000*a.p + a.l < 1000*b.p+b.l) + 3 * (1000*a.r+a.l < 1000*b.r+b.l) + 2 * (1000*a.s+a.l < 1000*b.s+b.l)
    score_a = 4 * (a.p > b.p) + 3 * (a.r > b.r) + 2 * (a.s > b.s)
    score_b = 4 * (a.p < b.p) + 3 * (a.r < b.r) + 2 * (a.s < b.s)
    melee_win = sign(score_a - score_b)
    if a_finds && b_finds
        tiebreaker = 4 * melee_win + 2 * sign(a.l - b.l) + sign(a.h - b.h)
        return sign(tiebreaker)
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        tiebreaker = 4 * sign(a.l - b.l) + 2 * melee_win + sign(a.f - b.f)
        return sign(tiebreaker)
    end
end


eval_battle (generic function with 1 method)

In [4]:
tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)
jobs = CSV.read("jobs.csv", DataFrame)

const mult_noun = 1
const mult_adj = 2
const mult_job = 3


function random_name_and_stat()::Spud
    vp = [0, 0, 0, 0, 0, 0]
    #nametype = rand([1,1,1,1,1,2,2,2,3])
    name = ""
    noun = ""
    adj = ""
    noun_i = rand(1:nrow(nouns))
    noun = nouns[noun_i, :noun]
    vp[1] = vp[1] + nouns[noun_i, :H] * mult_noun
    vp[2] = vp[2] + nouns[noun_i, :F] * mult_noun
    vp[3] = vp[3] + nouns[noun_i, :L] * mult_noun
    vp[4] = vp[4] + nouns[noun_i, :P] * mult_noun
    vp[5] = vp[5] + nouns[noun_i, :R] * mult_noun
    vp[6] = vp[6] + nouns[noun_i, :S] * mult_noun
    adj_i = rand(1:nrow(adjectives))
    adj = adjectives[adj_i, :adjective]
    vp[1] = vp[1] + adjectives[adj_i, :H] * mult_adj
    vp[2] = vp[2] + adjectives[adj_i, :F] * mult_adj
    vp[3] = vp[3] + adjectives[adj_i, :L] * mult_adj
    vp[4] = vp[4] + adjectives[adj_i, :P] * mult_adj
    vp[5] = vp[5] + adjectives[adj_i, :R] * mult_adj
    vp[6] = vp[6] + adjectives[adj_i, :S] * mult_adj
    job_i = rand(1:nrow(jobs))
    job = jobs[job_i, :job]
    vp[1] = vp[1] + jobs[job_i, :H] * mult_job
    vp[2] = vp[2] + jobs[job_i, :F] * mult_job
    vp[3] = vp[3] + jobs[job_i, :L] * mult_job
    vp[4] = vp[4] + jobs[job_i, :P] * mult_job
    vp[5] = vp[5] + jobs[job_i, :R] * mult_job
    vp[6] = vp[6] + jobs[job_i, :S] * mult_job
    name = string(adj, " ", job, " ", noun)
    Spud(name, vp[1], vp[2], vp[3], vp[4], vp[5], vp[6])
end

random_name_and_stat (generic function with 1 method)

In [5]:
function rand_rename(a::Spud, n_tries::Int = 100)::Spud
    best_score = 0.0
    best_b = random_name_and_stat()
    for ii in 1:n_tries
        b = random_name_and_stat()
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.p^2 + b.r^2 + b.s^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.p * b.p + a.r * b.r + a.s * b.s)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Spud(best_b.name, a.h, a.f, a.l, a.p, a.r, a.s)
end

rand_rename (generic function with 2 methods)

In [126]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end


function pick_best(as::Array{Spud}, bs::Array{Spud})::Spud
    bestscore = -999
    bestf = as[1]
    for ii in 1:length(as)
        score = eval_battle_list(as[ii], bs)
        if score > bestscore
            bestscore = score
            bestf = as[ii]
        end
    end
    return bestf
end

function pick_best_rdmly(as::Array{Spud}, bs::Array{Spud}, ntries::Int)::Spud
    bestscore = -999
    bestf = rand(as)
    for ii in 1:ntries
        f = rand(as)
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function pick_best_rdmly_g(g::Function, bs::Array{Spud}, ntries::Int)::Spud
    bestscore = -999
    bestf = g()
    for ii in 1:ntries
        f = g()
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

function eval_team_battle(as::Array{Spud}, bs::Array{Spud})::Int
    a_i = 1
    b_i = 1
    while (a_i <= length(as)) && (b_i <= length(bs))
        res = eval_battle(as[a_i], bs[b_i])
        if res == 1
            b_i = b_i + 1
        else
            a_i = a_i + 1
            if res == 0
                b_i = b_i + 1
            end
        end
    end
    a_out = (a_i > length(as))
    b_out = (b_i > length(as))
    if a_out
        if b_out
            return 0
        else
            return -1
        end
    else
        return 1
    end
end

function compare_generator(f1, f2, limit)
    a_i = 1
    b_i = 1
    f_a = f1()
    f_b = f2()
    while (a_i < limit) && (b_i < limit)
        res = eval_battle(f_a, f_b)
        if res != -1
            b_i = b_i + 1
            f_b = f2()
        end
        if res != 1
            a_i = a_i + 1
            f_a = f1()
        end
    end
    return (a_i/limit, b_i/limit)
end

function random_team(f::Function, team_size::Int)::Array{Spud}
    team = Array{Spud}(undef, team_size)
    for i in 1:team_size
        team[i] = f()
    end
    return team
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s])
    end
    return as
end

function cost(a::Spud)::Int64
    return cost(a.h, a.f, a.l, a.p, a.r, a.s)
end

cost (generic function with 2 methods)

In [7]:
function upgrade_spud(sp::Spud)::Spud
    h = sp.h
    f = sp.f
    l = sp.l
    p = sp.p
    r = sp.r
    s = sp.s    
    check_h = (h == MXV) || (cost(h+1,f,l,p,r,s) > MXS)
    check_f = (f == MXV) || (cost(h,f+1,l,p,r,s) > MXS)
    check_l = (l == MXV) || (cost(h,f,l+1,p,r,s) > MXS)
    check_prs = (p+r+s == 3*MXV) || (cost(h,f,l,p+1,r,s) > MXS)
    while !(check_h && check_f && check_l && check_prs)
        rand_i = rand(1:4)
        if rand_i == 1 && !check_h
            h = h+1
        end
        if rand_i == 2 && !check_f
            f = f+1
        end
        if rand_i == 3 && !check_l
            l = l+1
        end
        if rand_i == 4 && !check_prs
            rand_j = rand([1,1,1,1,1,2,2,2,2,3,3,3])
            if rand_j == 1 && p <= MXV
                p += 1
            end
            if rand_j == 2 && r <= MXV
                r += 1
            end
            if rand_j == 3 && s <= MXV
                s += 1
            end
        end            
        check_h = (h == MXV) || (cost(h+1,f,l,p,r,s) > MXS)
        check_f = (f == MXV) || (cost(h,f+1,l,p,r,s) > MXS)
        check_l = (l == MXV) || (cost(h,f,l+1,p,r,s) > MXS)
        check_prs = (p+r+s == 3*MXV) || (cost(h,f,l,p+1,r,s) > MXS)
    end
    return Spud(sp.name, h, f, l, p, r, s)
end



function pick_best_library(bs::Array{Spud})::Spud
    bestscore = -999
    bestf = Spud("",MNV,MNV,MNV,MNV,MNV,MNV)
    df = spuds_to_df(bs)
    hrange = vcat([MNV], df.h, df.h .+ 1, df.f, df.f .+ 1)
    frange = vcat([MNV], df.f, df.f .+ 1, df.h, df.h .+ 1)
    lrange = vcat([MNV], df.l, df.l .+ 1)
    prange = vcat([MNV], df.p, df.p .+ 1)
    rrange = vcat([MNV], df.r, df.r .+ 1)
    srange = vcat([MNV], df.s, df.s .+ 1)
    hrange = sort(unique(hrange))
    frange = sort(unique(frange))
    lrange = sort(unique(lrange))
    prange = sort(unique(prange))
    rrange = sort(unique(rrange))
    srange = sort(unique(srange))
    for h in hrange
        if (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
            for f in frange
                if (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                    for l in lrange
                        if (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                            for p in prange
                                if (cost(h, f, l, p, MNV, MNV) <= MXS)
                                    for r in rrange
                                        if (cost(h, f, l, p, r, MNV) <= MXS)
                                            for s in srange
                                                if (cost(h,f,l,p,r,s) <= MXS)
                                                    ff = Spud("",h,f,l,p,r,s)
                                                    score = eval_battle_list(ff, bs)
                                                    if score > bestscore
                                                        bestscore = score
                                                        bestf = ff
                                                    end
                                                end
                                            end
                                        end
                                    end                        
                                end
                            end                        
                        end
                    end
                end
            end
        end
    end
    return upgrade_spud(bestf)
end



pick_best_library (generic function with 1 method)

In [8]:
function pick_ok_counter(bs::Array{Spud})::Spud
    bestscore = -999
    bestf = Spud("",MNV,MNV,MNV,MNV,MNV,MNV)
    df = spuds_to_df(bs)
    hrange = vcat([MNV], df.f .+ 1)
    frange = vcat([MNV], df.h .+ 1)
    lrange = vcat([MNV], df.l .+ 1)
    prange = vcat([MNV], df.p .+ 1)
    rrange = vcat([MNV], df.r .+ 1)
    srange = vcat([MNV], df.s .+ 1)
    hrange = sort(unique(hrange))
    frange = sort(unique(frange))
    lrange = sort(unique(lrange))
    prange = sort(unique(prange))
    rrange = sort(unique(rrange))
    srange = sort(unique(srange))
    for h in hrange
        if (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
            for f in frange
                if (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                    for l in lrange
                        if (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                            for p in prange
                                if (cost(h, f, l, p, MNV, MNV) <= MXS)
                                    for r in rrange
                                        if (cost(h, f, l, p, r, MNV) <= MXS)
                                            for s in srange
                                                if (cost(h,f,l,p,r,s) <= MXS)
                                                    ff = Spud("",h,f,l,p,r,s)
                                                    score = eval_battle_list(ff, bs)
                                                    if score > bestscore
                                                        bestscore = score
                                                        bestf = ff
                                                    end
                                                end
                                            end
                                        end
                                    end                        
                                end
                            end                        
                        end
                    end
                end
            end
        end
    end
    return upgrade_spud(bestf)
end


pick_ok_counter (generic function with 1 method)

In [365]:
function pick_best_library2(bs::Array{Spud}, w::Vector{Float64})::Spud
    bestscore = -999.9
    bestf = Spud("",MNV,MNV,MNV,MNV,MNV,MNV)
    df = spuds_to_df(bs)
    hrange = vcat([MNV], df.h, df.h .+ 1, df.f, df.f .+ 1)
    frange = vcat([MNV], df.f, df.f .+ 1, df.h, df.h .+ 1)
    lrange = vcat([MNV], df.l, df.l .+ 1)
    prange = vcat([MNV], df.p, df.p .+ 1)
    rrange = vcat([MNV], df.r, df.r .+ 1)
    srange = vcat([MNV], df.s, df.s .+ 1)
    hrange = sort(unique(hrange))
    frange = sort(unique(frange))
    lrange = sort(unique(lrange))
    prange = sort(unique(prange))
    rrange = sort(unique(rrange))
    srange = sort(unique(srange))
    for h in hrange
        if (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
            for f in frange
                if (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                    for l in lrange
                        if (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                            for p in prange
                                if (cost(h, f, l, p, MNV, MNV) <= MXS)
                                    for r in rrange
                                        if (cost(h, f, l, p, r, MNV) <= MXS)
                                            for s in srange
                                                if (cost(h,f,l,p,r,s) <= MXS)
                                                    ff = Spud("",h,f,l,p,r,s)
                                                    score = eval_battle_list2(ff, bs, w) + 0.00001 * rand()
                                                    if score > bestscore
                                                        bestscore = score
                                                        bestf = ff
                                                    end
                                                end
                                            end
                                        end
                                    end                        
                                end
                            end                        
                        end
                    end
                end
            end
        end
    end
    return upgrade_spud(bestf)
end

pick_best_library2 (generic function with 1 method)

In [10]:
function subset_library(bs::Array{Spud}, n_hits::Int, w::Vector{Float64}, thres::Float64)::Array{Spud}
    bestf = Array{Spud}(undef, n_hits)
    spud_i = 0
    df = spuds_to_df(bs)
    hrange = vcat([MNV], df.h, df.h .+ 1, df.f, df.f .+ 1)
    frange = vcat([MNV], df.f, df.f .+ 1, df.h, df.h .+ 1)
    lrange = vcat([MNV], df.l, df.l .+ 1)
    prange = vcat([MNV], df.p, df.p .+ 1)
    rrange = vcat([MNV], df.r, df.r .+ 1)
    srange = vcat([MNV], df.s, df.s .+ 1)
    hrange = sort(unique(hrange))
    frange = sort(unique(frange))
    lrange = sort(unique(lrange))
    prange = sort(unique(prange))
    rrange = sort(unique(rrange))
    srange = sort(unique(srange))
    for h in hrange
        if (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
            for f in frange
                if (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                    for l in lrange
                        if (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                            for p in prange
                                if (cost(h, f, l, p, MNV, MNV) <= MXS)
                                    for r in rrange
                                        if (cost(h, f, l, p, r, MNV) <= MXS)
                                            for s in srange
                                                if (cost(h,f,l,p,r,s) <= MXS)
                                                    ff = Spud("",h,f,l,p,r,s)
                                                    score = eval_battle_list2(ff, bs, w)
                                                    if score >= thres && spud_i < n_hits
                                                        spud_i += 1
                                                        bestf[spud_i] = upgrade_spud(ff)
                                                    end
                                                end
                                            end
                                        end
                                    end                        
                                end
                            end                        
                        end
                    end
                end
            end
        end
    end
    return unique(bestf[1:spud_i])
end

subset_library (generic function with 1 method)

In [11]:
# form initial library by subsampling indices
library = Array{Spud}(undef, 1000)
spud_i = 0
ss_prob = 0.01

0.01

In [12]:

hrange = MNV:MXV
frange = MNV:MXV
lrange = MNV:MXV
prange = MNV:MXV
rrange = MNV:MXV
srange = MNV:MXV

for h in hrange
    if (cost(h, MXV, MXV, MXV, MXV, MXV) >= MXS) && (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
        for f in frange
            if (cost(h, f, MXV, MXV, MXV, MXV) >= MXS) && (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                for l in lrange
                    if (cost(h, f, l, MXV, MXV, MXV) >= MXS) && (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                        for p in prange
                            if (cost(h, f, l, p, MXV, MXV) >= MXS) && (cost(h, f, l, p, MNV, MNV) <= MXS)
                                for r in rrange
                                    if (cost(h, f, l, p, r, MXV) >= MXS) && (cost(h, f, l, p, r, MNV) <= MXS)
                                        for s in srange
                                            if rand() < ss_prob && (cost(h,f,l,p,r,s) <= MXS)
                                                check_f = (h == MXV) || (cost(h+1,f,l,p,r,s) > MXS)
                                                check_h = (f == MXV) || (cost(h,f+1,l,p,r,s) > MXS)
                                                check_l = (l == MXV) || (cost(h,f,l+1,p,r,s) > MXS)
                                                check_prs = (p+r+s == 3*MXV) || (cost(h,f,l,p+1,r,s) > MXS)
                                                if check_h && check_f && check_l && check_prs
                                                    spud_i += 1
                                                    #randname = rand_rename(Spud(" ",h,f,l,p,r,s)).name
                                                    #name = string("#", @sprintf("%i", spud_i), ". ", randname)
                                                    name = ""
                                                    library[spud_i] = Spud(name,h,f,l,p,r,s)
                                                end
                                            end
                                        end
                                    end
                                end                        
                            end
                        end                        
                    end
                end
            end
        end
    end
end


In [13]:
library = unique(library[1:spud_i])
n_spuds = length(library)

724

In [14]:
# for i in 1:n_spuds
#     ff = library[i]
#     randname = rand_rename(ff).name
#     name = string("#", @sprintf("%i", i), ". ", randname)
#     library[i] = Spud(name,ff.h,ff.f,ff.l,ff.p,ff.r,ff.s)
# end
# id_no = Dict(library[i] => i for i in 1:n_spuds)

In [15]:
# Check that there are no ties in spudland
for iter in 1:10000
    i = rand(1:n_spuds)
    j = rand(1:n_spuds)
    if i != j && eval_battle(library[i], library[j]) == 0
        println(library[i])
        println(library[j])
        println()
    end
end

## Compute Nash env

In [16]:
function ffp(nash_env, nits)
    n_nash = length(nash_env)
    i_lose = Array{Int}(undef, (n_nash, n_nash))
    n_lose = Array{Int}(undef, n_nash)
    for i in 1:n_nash
        n_lose[i] = 0
        ff = nash_env[i]
        for j in 1:n_nash
            if eval_battle(ff, nash_env[j]) ==-1
                n_lose[i] += 1
                i_lose[i, n_lose[i]] = j
            end
        end
    end
    counts = [0 for i in 1:n_nash]
    wins = [0 for i in 1:n_nash]
    for i in 1:n_nash
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    for iter in 1:nits
        wc = 2 .* wins .+ counts
        ind_winners = findall(wc .== maximum(wc))
        i = rand(ind_winners)
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    return counts
end


ffp (generic function with 1 method)

In [17]:
nash_env = [rand_rename(s) for s in library]
spuds_to_df(nash_env)

Row,name,h,f,l,p,r,s
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,Vigorous System designer Tashmijab,1,2,2,15,17,16
2,Bewitching System designer Batata vada,1,2,4,19,8,20
3,Irresistible Technical engineer Tashmijab,1,2,6,14,15,17
4,Virtuoso System designer Boulangère potatoes,1,2,6,16,17,13
5,Driven System designer French fries,1,2,6,18,10,18
6,Logical System designer Rappie pie,1,2,8,11,19,15
7,Tenacious Coast guard Kouign patatez,1,2,8,19,6,20
8,Self-reliant Paleontologist Rappie pie,1,2,8,20,18,7
9,Eager System designer Okroshka,1,2,10,17,15,12


In [18]:
counts = ffp(nash_env, 1000);

In [19]:
nash_env = nash_env[counts .> 10]
spuds_to_df(nash_env)

Row,name,h,f,l,p,r,s
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,Compassionate Joker Papas chorreadas,3,8,9,1,3,2
2,Daring Gondolier Rewena bread,4,5,13,1,1,3
3,Inattentive Magnate Potato babka,5,1,15,3,10,7
4,Sweet Party-leader Potato babka,5,2,15,3,2,2
5,Untroubled Miller Maluns,5,6,2,3,4,3
6,Objective Nanny (also nursemaid) Aligot,5,7,8,1,1,1
7,Relaxed Auctioneer Pommes Anna,6,1,13,5,1,10
8,Sage Housekeeper Pommes Anna,6,6,4,1,3,2
9,Unimposing Housekeeper Chips and dip,7,5,5,3,1,2


In [20]:
counts = ffp(nash_env, 10000)

10-element Vector{Int64}:
 2031
    1
    1
    1
 1986
    1
 2001
    1
 1959
 2028

In [21]:
nash_env = nash_env[counts .> 10]
spuds_to_df(nash_env)

Row,name,h,f,l,p,r,s
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,Compassionate Joker Papas chorreadas,3,8,9,1,3,2
2,Untroubled Miller Maluns,5,6,2,3,4,3
3,Relaxed Auctioneer Pommes Anna,6,1,13,5,1,10
4,Unimposing Housekeeper Chips and dip,7,5,5,3,1,2
5,Peaceful Copywriter Älplermagronen,8,1,10,2,6,4


### Loop this

In [22]:
counts = ffp(nash_env, 100)

5-element Vector{Int64}:
 20
 20
 22
 18
 25

In [23]:
#counts = ffp(nash_env, 10000)

In [24]:
spuds_to_df(nash_env)

Row,name,h,f,l,p,r,s
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,Compassionate Joker Papas chorreadas,3,8,9,1,3,2
2,Untroubled Miller Maluns,5,6,2,3,4,3
3,Relaxed Auctioneer Pommes Anna,6,1,13,5,1,10
4,Unimposing Housekeeper Chips and dip,7,5,5,3,1,2
5,Peaceful Copywriter Älplermagronen,8,1,10,2,6,4


In [25]:
# # don't run this every iteration!!
# nash_env = nash_env[counts .> 10]
# counts = ffp(nash_env, 10000)

In [26]:
ff = rand_rename(pick_best_library2(nash_env, counts./sum(counts)))

Spud("Perfectionist Stewardess (or Steward) Chapalele", 2, 8, 6, 4, 1, 4)

In [27]:
ev = eval_battle_list2(ff, nash_env, counts./sum(counts))

1.0

In [28]:
bestf = subset_library(nash_env, 500, counts./sum(counts), ev-0.05)

34-element Vector{Spud}:
 Spud("", 1, 8, 4, 5, 2, 4)
 Spud("", 1, 8, 4, 6, 1, 4)
 Spud("", 2, 7, 11, 4, 1, 4)
 Spud("", 2, 7, 11, 3, 5, 1)
 Spud("", 2, 7, 15, 4, 1, 3)
 Spud("", 2, 7, 11, 4, 2, 3)
 Spud("", 2, 7, 11, 4, 4, 1)
 Spud("", 2, 7, 11, 5, 1, 3)
 Spud("", 2, 8, 2, 1, 5, 4)
 Spud("", 2, 8, 2, 4, 5, 1)
 Spud("", 2, 8, 2, 3, 5, 2)
 Spud("", 2, 8, 2, 3, 6, 1)
 Spud("", 2, 9, 5, 4, 1, 3)
 ⋮
 Spud("", 2, 8, 2, 5, 2, 3)
 Spud("", 2, 8, 2, 5, 4, 1)
 Spud("", 2, 8, 6, 3, 5, 1)
 Spud("", 3, 8, 4, 4, 1, 3)
 Spud("", 2, 8, 6, 4, 2, 3)
 Spud("", 2, 8, 6, 4, 4, 1)
 Spud("", 2, 8, 6, 4, 1, 4)
 Spud("", 2, 8, 6, 5, 1, 3)
 Spud("", 2, 8, 10, 4, 1, 3)
 Spud("", 3, 8, 1, 3, 5, 1)
 Spud("", 3, 8, 1, 5, 1, 3)
 Spud("", 4, 8, 1, 4, 1, 3)

In [29]:
cc = ffp(bestf, 10000)
println(maximum(cc))
ff2 = rand_rename(bestf[cc .== maximum(cc)][1])

10001


Spud("Principled Shoeshiner Boulangère potatoes", 1, 8, 4, 6, 1, 4)

In [30]:
eval_battle_list2(ff2, nash_env, counts./sum(counts))

1.0

In [31]:
append!(nash_env, [ff2])
spuds_to_df(nash_env)

Row,name,h,f,l,p,r,s
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,Compassionate Joker Papas chorreadas,3,8,9,1,3,2
2,Untroubled Miller Maluns,5,6,2,3,4,3
3,Relaxed Auctioneer Pommes Anna,6,1,13,5,1,10
4,Unimposing Housekeeper Chips and dip,7,5,5,3,1,2
5,Peaceful Copywriter Älplermagronen,8,1,10,2,6,4
6,Principled Shoeshiner Boulangère potatoes,1,8,4,6,1,4


### loop

In [32]:
counts = ffp(nash_env, 100000)
println(maximum([eval_battle_list2(ff, nash_env, counts./sum(counts)) for ff in nash_env]))
ff = rand_rename(pick_best_library2(nash_env, counts./sum(counts)))
ev = eval_battle_list2(ff, nash_env, counts./sum(counts))
println(ev)

4.9997000179989205e-5
0.999980001199928


In [33]:
bestf = subset_library(nash_env, 900, counts./sum(counts), ev-0.05)

797-element Vector{Spud}:
 Spud("", 6, 4, 2, 5, 4, 7)
 Spud("", 3, 5, 5, 1, 3, 10)
 Spud("", 5, 3, 6, 2, 4, 12)
 Spud("", 9, 2, 6, 2, 4, 8)
 Spud("", 4, 4, 6, 2, 3, 10)
 Spud("", 2, 4, 8, 3, 4, 12)
 Spud("", 3, 4, 8, 5, 5, 6)
 Spud("", 3, 4, 6, 1, 5, 11)
 Spud("", 5, 4, 3, 1, 4, 11)
 Spud("", 8, 3, 3, 1, 7, 9)
 Spud("", 3, 5, 1, 1, 5, 10)
 Spud("", 6, 3, 3, 3, 5, 13)
 Spud("", 5, 4, 4, 3, 7, 5)
 ⋮
 Spud("", 4, 2, 10, 9, 3, 14)
 Spud("", 3, 3, 8, 8, 2, 12)
 Spud("", 3, 3, 13, 10, 5, 2)
 Spud("", 2, 4, 10, 10, 5, 3)
 Spud("", 5, 3, 7, 7, 4, 5)
 Spud("", 3, 3, 12, 7, 6, 5)
 Spud("", 7, 1, 10, 9, 3, 11)
 Spud("", 2, 4, 8, 10, 5, 4)
 Spud("", 4, 3, 10, 8, 5, 3)
 Spud("", 6, 2, 7, 8, 8, 7)
 Spud("", 4, 3, 6, 7, 7, 7)
 Spud("", 5, 2, 7, 9, 7, 11)

In [34]:
ff

Spud("Transcendent Auctioneer Batata vada", 2, 5, 15, 6, 3, 3)

In [35]:
ff = rand_rename(pick_best_library2(nash_env, counts./sum(counts)))
ff

Spud("Goal-focused Barman Okroshka", 2, 5, 15, 3, 4, 5)

In [36]:
eval_battle_list2(ff, nash_env, counts./sum(counts))

0.999980001199928

In [37]:
ev

0.999980001199928

In [38]:
ev

0.999980001199928

In [39]:
println(length(bestf))
cc = ffp(bestf, 10000)
println(maximum(cc))

797
2076


In [40]:
ff2 = rand_rename(bestf[cc .== maximum(cc)][1])

Spud("Exquisite Auctioneer Rewena bread", 5, 1, 15, 9, 4, 7)

In [41]:
println(eval_battle_list2(ff2, nash_env, counts./sum(counts)))

0.9999400035997841


In [42]:
append!(nash_env, [ff2])

7-element Vector{Spud}:
 Spud("Compassionate Joker Papas chorreadas", 3, 8, 9, 1, 3, 2)
 Spud("Untroubled Miller Maluns", 5, 6, 2, 3, 4, 3)
 Spud("Relaxed Auctioneer Pommes Anna", 6, 1, 13, 5, 1, 10)
 Spud("Unimposing Housekeeper Chips and dip", 7, 5, 5, 3, 1, 2)
 Spud("Peaceful Copywriter Älplermagronen", 8, 1, 10, 2, 6, 4)
 Spud("Principled Shoeshiner Boulangère potatoes", 1, 8, 4, 6, 1, 4)
 Spud("Exquisite Auctioneer Rewena bread", 5, 1, 15, 9, 4, 7)

In [425]:
nash_env = df_to_spuds(DataFrame(CSV.File("temp_factored.csv")))

1800-element Vector{Spud}:
 Spud("Courageous Nanny (also nursemaid) Clapshot", 5, 7, 8, 1, 1, 1)
 Spud("Tolerant Joker Tartiflette", 5, 8, 7, 1, 1, 1)
 Spud("Outstanding Ferryman Nikujaga", 5, 11, 2, 1, 1, 1)
 Spud("Mysterious Gladiator Stoemp", 6, 5, 9, 1, 1, 1)
 Spud("Selfless Nanny (also nursemaid) Tartiflette", 6, 7, 6, 1, 1, 1)
 Spud("Engaging Ferryman Rewena bread", 6, 10, 1, 1, 1, 1)
 Spud("Credible Gladiator Brændende kærlighed", 7, 3, 10, 1, 1, 1)
 Spud("Judicious Gladiator Gamjajeon", 7, 5, 7, 1, 1, 1)
 Spud("Expressive Ferryman Rewena bread", 7, 7, 4, 1, 1, 1)
 Spud("Aloof Ferryman Tartiflette", 7, 9, 1, 1, 1, 1)
 Spud("Passionless Copywriter Gamjajeon", 8, 4, 7, 1, 1, 1)
 Spud("Compassionate Ferryman Suan la tu dou si", 8, 8, 1, 1, 1, 1)
 Spud("Instinctive Chimney sweep Chapalele", 9, 6, 3, 1, 1, 1)
 ⋮
 Spud("Absorbed Impersonator Stoemp", 7, 1, 11, 3, 8, 5)
 Spud("Proactive Cardinal Batata vada", 7, 1, 11, 5, 4, 7)
 Spud("Steely Cardinal Tashmijab", 7, 1, 11, 6, 8, 2)
 Spu

In [426]:
nits = 50000000
tol = 0.005
println(Dates.format(now(), "HH:MM"))
for ii in 1:20
    #println()
    print(ii)
    counts = ffp(nash_env, nits)
    w = counts./sum(counts)
    #println(maximum([eval_battle_list2(ff, nash_env, counts./sum(counts)) for ff in nash_env]))
    ff = rand_rename(pick_best_library2(nash_env, w))
    #ev = eval_battle_list2(ff, nash_env, w)
    #println(ev)
    #bestf = subset_library(nash_env, 900, counts./sum(counts), tol)
    #println(length(bestf))
    #cc = ffp(bestf, nits)
    #println(maximum(cc))
    #ff2 = rand_rename(bestf[cc .== maximum(cc)][1])
    #println(ff2)
    #println(eval_battle_list2(ff2, nash_env, counts./sum(counts)))
    #append!(nash_env, [ff2])
    append!(nash_env, [ff])
    println(ff)
end
println()
println(Dates.format(now(), "HH:MM"))
counts = ffp(nash_env, nits)
#ff = pick_best_library2(nash_env, counts./sum(counts))
#eval_battle_list2(ff, nash_env, counts./sum(counts))    
#nash_env = nash_env[counts .> 10]
#counts = ffp(nash_env, nits);
println(maximum([eval_battle_list2(ff, nash_env, counts./sum(counts)) for ff in nash_env]))
println(length(nash_env))
println(length(unique([Spud("", s.h, s.f, s.l, s.p, s.r, s.s) for s in nash_env])))    
ff = pick_best_library2(nash_env, counts./sum(counts))
eval_battle_list2(ff, nash_env, counts./sum(counts))

13:24
1Spud("Boring Magnate Boulangère potatoes", 4, 4, 10, 1, 7, 3)
2Spud("Prudent Dictator Gamjajeon", 7, 1, 11, 6, 8, 2)
3Spud("Impressive Stewardess (or Steward) Pommes Anna", 4, 4, 10, 3, 2, 6)
4Spud("Imaginative Real estate investor Pommes Anna", 4, 2, 15, 1, 8, 7)
5Spud("Passionate Impersonator Batata vada", 5, 1, 18, 1, 2, 2)
6Spud("Unattractive Teacher Kouign patatez", 3, 12, 1, 3, 1, 1)
7Spud("Show-off Housekeeper Rewena bread", 8, 5, 3, 1, 3, 3)
8Spud("Unpleased Gondolier Potato babka", 4, 4, 10, 3, 6, 2)
9

LoadError: InterruptException:

In [427]:
println()
println(Dates.format(now(), "HH:MM"))
counts = ffp(nash_env, nits)
#ff = pick_best_library2(nash_env, counts./sum(counts))
#eval_battle_list2(ff, nash_env, counts./sum(counts))    
#nash_env = nash_env[counts .> 10]
#counts = ffp(nash_env, nits);
println(maximum([eval_battle_list2(ff, nash_env, counts./sum(counts)) for ff in nash_env]))
println(length(nash_env))
println(length(unique([Spud("", s.h, s.f, s.l, s.p, s.r, s.s) for s in nash_env])))    
ff = pick_best_library2(nash_env, counts./sum(counts))
eval_battle_list2(ff, nash_env, counts./sum(counts))


13:56
0.001278253778343559
1808
1801


0.001278253778343559

In [369]:
#CSV.write("spudsA_nash_0_00382.csv", spuds_to_df(nash_env))

#### Nash env explotability very delicate!

In [388]:
#nash_env = df_to_spuds(DataFrame(CSV.File("spudsA_nash_0_00192.csv")));

In [390]:
counts = ffp(nash_env, nits)
println(maximum([eval_battle_list2(ff, nash_env, counts./sum(counts)) for ff in nash_env]))
println(length(nash_env))
println(length(unique([Spud("", s.h, s.f, s.l, s.p, s.r, s.s) for s in nash_env])))    
ff = pick_best_library2(nash_env, counts./sum(counts))
eval_battle_list2(ff, nash_env, counts./sum(counts))

0.0018508349055255803
1784
1774


0.0018508349055255803

In [391]:
nash_env = unique([Spud("", s.h, s.f, s.l, s.p, s.r, s.s) for s in nash_env]);

In [392]:
counts = ffp(nash_env, nits)
println(maximum([eval_battle_list2(ff, nash_env, counts./sum(counts)) for ff in nash_env]))
println(length(nash_env))
println(length(unique([Spud("", s.h, s.f, s.l, s.p, s.r, s.s) for s in nash_env])))    
ff = pick_best_library2(nash_env, counts./sum(counts))
eval_battle_list2(ff, nash_env, counts./sum(counts))

0.0018769835115623448
1774
1774


0.001873633808681957

In [393]:
nash_env2 = nash_env[counts .> 1]
length(nash_env2)

892

In [394]:
counts2 = ffp(nash_env2, nits)
ff = pick_best_library2(nash_env2, counts2./sum(counts2))
eval_battle_list2(ff, nash_env2, counts2./sum(counts2))

0.0025046882909023054

In [395]:
counts3 = counts[counts .> 1]
ff = pick_best_library2(nash_env2, counts3./sum(counts3))
eval_battle_list2(ff, nash_env2, counts3./sum(counts3))

0.001880066149049776

In [396]:
for i in 1:length(nash_env2)
    ff = nash_env2[i]
    nash_env2[i] = Spud(string("c", counts3[i]), ff.h, ff.f, ff.l, ff.p, ff.r, ff.s)
end

In [397]:
nash_env2 = nash_env2[sortperm(-counts3)]
nash_env2

892-element Vector{Spud}:
 Spud("c352167", 6, 1, 15, 2, 1, 1)
 Spud("c341656", 10, 6, 2, 1, 1, 1)
 Spud("c297990", 9, 6, 3, 1, 1, 1)
 Spud("c276652", 6, 10, 1, 1, 1, 1)
 Spud("c261422", 5, 11, 2, 1, 1, 1)
 Spud("c252779", 10, 5, 3, 2, 1, 1)
 Spud("c201180", 7, 7, 4, 1, 1, 1)
 Spud("c193968", 8, 7, 2, 2, 1, 1)
 Spud("c175435", 11, 3, 5, 2, 1, 1)
 Spud("c173597", 7, 9, 1, 1, 1, 1)
 Spud("c168894", 3, 12, 1, 1, 2, 2)
 Spud("c162311", 4, 11, 3, 2, 1, 1)
 Spud("c156334", 3, 10, 3, 3, 2, 1)
 ⋮
 Spud("c13", 3, 3, 15, 2, 7, 6)
 Spud("c12", 4, 2, 17, 2, 6, 4)
 Spud("c8", 6, 1, 13, 2, 8, 6)
 Spud("c6", 7, 3, 7, 1, 6, 3)
 Spud("c6", 6, 2, 11, 1, 6, 4)
 Spud("c6", 11, 1, 7, 7, 3, 2)
 Spud("c6", 5, 1, 16, 2, 6, 7)
 Spud("c5", 6, 4, 8, 3, 3, 1)
 Spud("c5", 6, 2, 11, 6, 2, 3)
 Spud("c3", 6, 1, 13, 6, 8, 2)
 Spud("c2", 5, 1, 18, 2, 1, 2)
 Spud("c2", 10, 1, 8, 5, 1, 4)

In [399]:
#CSV.write("spudsA_nash_0_00192_reduced.csv", spuds_to_df(nash_env2))

In [256]:
#nash_env

## Generate all HFL and PRS combos

In [428]:
df = spuds_to_df(nash_env);
df.t = df.p + df.r + df.s;
thfl = Matrix(df[:, [:t, :h, :f, :l]])
tprs = Matrix(df[:, [:t, :p, :r, :s]])
thfl = transpose(reduce(hcat, unique(eachrow(thfl))))
tprs = transpose(reduce(hcat, unique(eachrow(tprs))))
for i in [4, 3, 2, 1]
    thfl = thfl[sortperm(thfl[:, i]), :]
    tprs = tprs[sortperm(tprs[:, i]), :]
end
min_t = minimum(df.t)
max_t = maximum(df.t)
writedlm("temp_hfl.txt", thfl, ' ')
writedlm("temp_prs.txt", tprs, ' ')

# generate the product of all HFL and PRS combos
new_env = Array{Spud}(undef, 20000)
spud_i = 0
for t in min_t:max_t
    hfl = thfl[thfl[:, 1].==t, 2:end]
    prs = tprs[tprs[:, 1].==t, 2:end]
    for a in eachrow(hfl)
        for b in eachrow(prs)
            spud_i += 1
            new_env[spud_i] = Spud("", a[1], a[2], a[3], b[1], b[2], b[3])
        end
    end
end
new_env = new_env[1:spud_i]

1807-element Vector{Spud}:
 Spud("", 5, 7, 8, 1, 1, 1)
 Spud("", 5, 8, 7, 1, 1, 1)
 Spud("", 5, 11, 2, 1, 1, 1)
 Spud("", 6, 5, 9, 1, 1, 1)
 Spud("", 6, 7, 6, 1, 1, 1)
 Spud("", 6, 10, 1, 1, 1, 1)
 Spud("", 7, 3, 10, 1, 1, 1)
 Spud("", 7, 5, 7, 1, 1, 1)
 Spud("", 7, 7, 4, 1, 1, 1)
 Spud("", 7, 9, 1, 1, 1, 1)
 Spud("", 8, 4, 7, 1, 1, 1)
 Spud("", 8, 8, 1, 1, 1, 1)
 Spud("", 9, 6, 3, 1, 1, 1)
 ⋮
 Spud("", 7, 1, 11, 3, 8, 5)
 Spud("", 7, 1, 11, 5, 4, 7)
 Spud("", 7, 1, 11, 6, 8, 2)
 Spud("", 7, 1, 11, 7, 2, 7)
 Spud("", 7, 1, 11, 7, 7, 2)
 Spud("", 7, 1, 11, 7, 8, 1)
 Spud("", 7, 1, 11, 8, 1, 7)
 Spud("", 7, 1, 11, 8, 2, 6)
 Spud("", 7, 1, 11, 8, 7, 1)
 Spud("", 7, 1, 11, 9, 1, 6)
 Spud("", 7, 1, 11, 9, 6, 1)
 Spud("", 7, 1, 11, 10, 1, 5)

In [429]:
@time counts = ffp(new_env, nits);

241.582082 seconds (250.00 M allocations: 897.076 GiB, 19.77% gc time)


In [441]:
for i in 1:length(new_env)
    ff = new_env[i]
    new_env[i] = Spud(string("c", counts[i]), ff.h, ff.f, ff.l, ff.p, ff.r, ff.s)
end
new_env

1807-element Vector{Spud}:
 Spud("c33694", 5, 7, 8, 1, 1, 1)
 Spud("c59714", 5, 8, 7, 1, 1, 1)
 Spud("c657615", 5, 11, 2, 1, 1, 1)
 Spud("c96372", 6, 5, 9, 1, 1, 1)
 Spud("c196271", 6, 7, 6, 1, 1, 1)
 Spud("c677613", 6, 10, 1, 1, 1, 1)
 Spud("c210242", 7, 3, 10, 1, 1, 1)
 Spud("c189348", 7, 5, 7, 1, 1, 1)
 Spud("c558612", 7, 7, 4, 1, 1, 1)
 Spud("c407253", 7, 9, 1, 1, 1, 1)
 Spud("c102169", 8, 4, 7, 1, 1, 1)
 Spud("c104872", 8, 8, 1, 1, 1, 1)
 Spud("c719008", 9, 6, 3, 1, 1, 1)
 ⋮
 Spud("c10880", 7, 1, 11, 3, 8, 5)
 Spud("c1", 7, 1, 11, 5, 4, 7)
 Spud("c43663", 7, 1, 11, 6, 8, 2)
 Spud("c14177", 7, 1, 11, 7, 2, 7)
 Spud("c4124", 7, 1, 11, 7, 7, 2)
 Spud("c40487", 7, 1, 11, 7, 8, 1)
 Spud("c30729", 7, 1, 11, 8, 1, 7)
 Spud("c2883", 7, 1, 11, 8, 2, 6)
 Spud("c19403", 7, 1, 11, 8, 7, 1)
 Spud("c58168", 7, 1, 11, 9, 1, 6)
 Spud("c50203", 7, 1, 11, 9, 6, 1)
 Spud("c31823", 7, 1, 11, 10, 1, 5)

In [442]:
CSV.write("spudsA_full_factor.csv", spuds_to_df(new_env))

"spudsA_full_factor.csv"

In [430]:
new_env2 = new_env[counts .> 1];
#@time counts = ffp(new_env2, nits)
w = counts[counts .> 1]./sum(counts[counts .> 1]);

In [431]:
length(new_env2)

887

In [444]:
ff = pick_best_library2(new_env2, w)
eval_battle_list2(ff, new_env2, w)

0.00164447082708762

In [433]:
println(maximum([eval_battle_list2(ff, new_env2, w) for ff in new_env2]))

0.001266557531269538


In [434]:
#nash_copy = unique([Spud("", s.h, s.f, s.l, s.p, s.r, s.s) for s in nash_env])


In [435]:
scores = [eval_battle_list2(ff, new_env, counts./sum(counts)) for ff in new_env];

In [436]:
for i in sortperm(scores)[1:10]
    print(new_env[i])
    print(" ")
    println(scores[i])
end

Spud("", 1, 11, 1, 6, 1, 1) -0.10735936003272906
Spud("", 6, 4, 6, 3, 6, 1) -0.10032241434794659
Spud("", 7, 3, 6, 3, 2, 7) -0.08714045074411014
Spud("", 6, 4, 6, 7, 2, 1) -0.08572754180663968
Spud("", 2, 11, 6, 2, 1, 3) -0.0781464957856409
Spud("", 2, 10, 10, 2, 1, 3) -0.07665018986213709
Spud("", 1, 9, 1, 3, 6, 1) -0.0762437245517934
Spud("", 2, 6, 14, 3, 6, 1) -0.07540979469001838
Spud("", 7, 3, 6, 8, 2, 2) -0.0751820029224147
Spud("", 2, 12, 2, 2, 1, 3) -0.074707580068055


In [438]:
# for i in 1:length(new_env)
#     ff = new_env[i]
#     new_env[i] = Spud(string("s ", convert(Int, round(scores[i] * 100))), ff.h, ff.f, ff.l, ff.p, ff.r, ff.s)
# end
# new_env

In [439]:
#CSV.write("spudsA_full_factor_scores.csv", spuds_to_df(new_env))

## beware

In [445]:
#CSV.write("temp_factored.csv", spuds_to_df([rand_rename(s) for s in new_env2]))

"temp_factored.csv"

In [66]:
#CSV.write("temp.csv", spuds_to_df(nash_env))

"temp.csv"

## when would you be tempted to go far outside of nash equilibrium?

In [446]:
nash_env_df = DataFrame(CSV.File("spudsA_full_factor.csv"))
nash_env = df_to_spuds(nash_env_df)
counts = [parse(Int, s.name[2:end]) for s in nash_env];

In [448]:
cc = cumsum(counts);

In [503]:
nteam = 5
mms = Array{Float64}(undef, 1000)
for jj in 1:1000
    team = Array{Spud}(undef, nteam)
    for i in 1:nteam
        tmp = rand(1:sum(counts))
        ind = sum(cc .< tmp)+1
        team[i] = nash_env[ind]
        #println(counts[ind])
    end
    lib = subset_library(team, 900, [1/nteam for i in 1:nteam], 0.001)
    scores = [eval_battle_list(ff, team) for ff in lib]
    lib = lib[scores .== maximum(scores)]
    evs = [eval_battle_list2(ff, nash_env, counts./sum(counts)) for ff in lib]
    mm = maximum(evs)
    mms[jj] = mm
    if mm < -0.2
        println()
        println(spuds_to_df(team))
        println(mm)
        println(lib[evs .== mm])
        
    end
end


5×7 DataFrame
 Row │ name     h      f      l      p      r      s     
     │ String   Int64  Int64  Int64  Int64  Int64  Int64 
─────┼───────────────────────────────────────────────────
   1 │ c558612      7      7      4      1      1      1
   2 │ c242854      5      2     12      1      7      7
   3 │ c348869      9      5      2      1      3      3
   4 │ c15389       1      9      1      1      4      5
   5 │ c912227      6      1     15      2      1      1
-0.2767748973552108
Spud[Spud("", 1, 9, 1, 2, 7, 1)]

5×7 DataFrame
 Row │ name     h      f      l      p      r      s     
     │ String   Int64  Int64  Int64  Int64  Int64  Int64 
─────┼───────────────────────────────────────────────────
   1 │ c198043      5      1     18      3      1      1
   2 │ c225721      8      2      8      6      3      1
   3 │ c912227      6      1     15      2      1      1
   4 │ c215684      3     10      3      1      3      2
   5 │ c99493      10      4      3      1      3      3

In [504]:
sort(mms)

1000-element Vector{Float64}:
 -0.2767748973552108
 -0.23906130032460657
 -0.22879695127818175
 -0.22066668510599904
 -0.21175516716825818
 -0.20739516473874706
 -0.2016793313089685
 -0.18596045938899716
 -0.18573734745226259
 -0.17754476353224577
 -0.1753648423145975
 -0.1609076047991624
 -0.15762096357837432
  ⋮
  0.001649880373323649
  0.001649880373323649
  0.001649880373323649
  0.001649880373323649
  0.001649880373323649
  0.001649880373323649
  0.001649880373323649
  0.001649880373323649
  0.001649880373323649
  0.001649880373323649
  0.001649880373323649
  0.001649880373323649

In [505]:
mean(mms .< -0.05)

0.095

In [506]:
mean(mms .< -0.1)

0.051

In [507]:
mean(mms .< -0.15)

0.015